#  Optimization of Autonomous Bus Stops
## Game Theory and Multi-Objective Optimization


**Context:** An autonomous bus must determine its optimal stops in a city organized in a 10×10 grid. Ten passengers inside the bus have different preferences for the stops. What are the Pareto optima and the nash equilibria ?


In [2]:
#few libraries
import numpy as np
import matplotlib.pyplot as plt
from itertools import combinations
from scipy.spatial.distance import euclidean
import random
from typing import List, Tuple, Set
from dataclasses import dataclass

### Model
Model of the travellers and their preferences

In [30]:
@dataclass
class PointPreference:
    """ a preferred point and its value"""
    x: int
    y: int
    value: float
    
    def position(self) -> Tuple[int, int]:
        return (self.x, self.y)
    
    def __repr__(self):
        return f"PointPreference ({self.x},{self.y})={self.value:.2f}"

In [31]:
class Passenger:
    """
    Model of a traveller with his stop preferences
    """
    def __init__(self, id_passager: int, x_start: int, y_start: int, 
                 preferences: List[PointPreference]):
        self.id = id_passager
        self.x_start = x_start
        self.y_start = y_start
        # sort prefered points by value descending
        self.preferences = sorted(preferences, key=lambda p: p.value, reverse=True)
        
    def compute_point_utility(self, x: int, y: int, distance_max: float = 14.14) -> float:
        """
        Compute the utility of any point for this passenger. 
        The value decreases linearly with the distance to the nearest preferred point.
        
        Args:
            x, y: coordinates of the point to evaluate
            distance_max: distance at which the utility becomes zero
        """
        # is it a preferred point?
        for pref in self.preferences:
            if pref.x == x and pref.y == y:
                return pref.value
        
        # not a preferred point, compute utility based on distance to preferred points
        max_utility = 0
        for pref in self.preferences:
            #use euclidean distance because city is a grid
            dist = euclidean((x, y), (pref.x, pref.y))
            utility = max(0, pref.value * (1 - dist / distance_max))
            max_utility = max(max_utility, utility)
        
        return max_utility
    
    def compute_max_utility(self, stops: Set[Tuple[int, int]]) -> float:
        """
        Compute the utility for a given set of stops.
        Takes the best available stop.
        """
        if not stops:
            return 0.0
        
        utilites = [self.compute_point_utility(x, y) for x, y in stops]
        return max(utilites)
    
    def __repr__(self):
        points_str = ', '.join([str(p) for p in self.preferences])
        return f"Passager {self.id} @ ({self.x_start},{self.y_start}) Prefs: [{points_str}]"

In [32]:
# Example usage
p = Passenger(1, 0, 0, [PointPreference(2, 2, 10), PointPreference(5, 5, 5)])
print(p.compute_point_utility(2, 2))  # Should return 10
print(p.compute_point_utility(3, 3))  # Should return a value less than 10
print(p.compute_max_utility({(2, 2), (3, 3)}))  # Should return 10

10
8.99984896578989
10


Let's generate passengers..

In [33]:
def generate_random_passengers(nbp: int = 10, grid_size: int = 10,  seed: int = 42) -> List[Passenger]:
    """
    Generates np passengers with coherent random preferences.
    
    Args:
        np: nb of passengers to generate
        grid_size: Taille de la grille (10x10 par défaut)
        seed: Graine aléatoire pour reproductibilité
    """
    random.seed(seed)
    
    passengers = []
    
    for i in range(nbp):
        # Position de départ aléatoire
        x_start = random.randint(0, grid_size - 1)
        y_start = random.randint(0, grid_size - 1)
        
        # Générer 3 points préférés uniques
        choosen_points = set()
        preferences = []
        
        # Les valeurs décroissent : v1 > v2 > v3
        valeurs = sorted([random.uniform(50, 100), 
                         random.uniform(30, 70), 
                         random.uniform(10, 50)], reverse=True)
        
        for j, valeur in enumerate(valeurs):
            # Éviter les doublons
            while True:
                x = random.randint(0, grid_size - 1)
                y = random.randint(0, grid_size - 1)
                if (x, y) not in choosen_points:
                    choosen_points.add((x, y))
                    break
            
            preferences.append(PointPreference(x, y, valeur))
        
        passengers.append(Passenger(i, x_start, y_start, preferences))
    
    return passengers


In [34]:
# Example usage
passengers = generate_random_passengers(3, 10, 42)
for p in passengers: print(p)

Passager 0 @ (1,0) Prefs: [PointPreference (1,8)=87.08, PointPreference (1,9)=39.80, PointPreference (6,0)=15.58]
Passager 1 @ (0,1) Prefs: [PointPreference (3,8)=60.93, PointPreference (6,3)=50.21, PointPreference (7,9)=11.06]
Passager 2 @ (4,0) Prefs: [PointPreference (4,2)=87.94, PointPreference (3,5)=36.39, PointPreference (1,1)=26.90]


In [ ]:
#function that can be used or not to compute utilities of a set of stops for all passengers, 
# just given in case you need it later
def compute_utilities(stops: Set[Tuple[int, int]], 
                     passengers: List[Passenger]) -> np.ndarray:
    """
    compute utilities vector for all passengers   
    Args:
        stops:  selected stops set 
        passengers:  passengers list
    Returns:
        Array of len(passengers) with  individual utilities
    """
    return np.array([p.compute_utility_configuration(stops) for p in passengers])


In [41]:
# Example usage
passengers = generate_random_passengers(5, 10, 42)
stops = {(2, 2), (5, 5), (7, 7)}
max_utilities = compute_utilities(stops, passengers)
print("Max interest of the points for the passenger")
i = 0
for s in stops:
    print(f"{s} utility: {max_utilities[i]}")
    i+=1


Max interest of the points for the passengers
(5, 5) utility: 56.28632096112048
(7, 7) utility: 45.39490544321417
(2, 2) utility: 75.50184383647587


---
### Paretos points


To find the pareto points (front), we have to check all the possible deal points (10x10 here).
So each passenger has to valuate all of the 100 points.
Next, we have to find the list of Pareto optimal points (a pareto optimal point is not dominated by any other point).


Define the function that determinate if a point is a Pareto point or not.

In [ ]:
def is_pareto_optimal(stop: Tuple[int, int], all_points: List[Set[Tuple[int, int]]], passengers: List[Passenger]) -> bool:
    """"
    Determine if a given stop_set is Pareto optimal among all_sets for the given passengers.
    Args:
        stop: The stop to evaluate.
        all_points: List of all possible  stops.
        passengers: List of passengers.
    """
    is_pareto_optimal = False
    #TODO: implement the function
    return is_pareto_optimal